In [1]:
import pandas as pd
import numpy as np

In [2]:
# Retrieve the Bridge dataframe
df = pd.read_excel('BMMS_overview.xlsx')

In [3]:
# Create a subset of the dataframe containing only bridges on road N1
df_N1 = df[df['road']=='N1']
# Sort the dataframe based on km, so the bridges follow the correct order
df_N1_sorted = df_N1.sort_values(by='km').reset_index().drop(labels='index', axis=1)
# Create a list of the indexes
index = df_N1_sorted.index

In [4]:
def create_links(df, indexes):
    # Create an empty dictionary
    inserting_links = {}
    # Loop over all the bridges, skipping the first one
    for i in index[1:]:
        # Determine the latitude of the bridge
        lat = df.iloc[i, df.columns.get_indexer(['lat'])].values
        # Determine the longitude of the bridge
        lon = df.iloc[i, df.columns.get_indexer(['lon'])].values
        # Determine the latitude of the previous bridge
        lat_prev = df.iloc[i-1, df.columns.get_indexer(['lat'])].values
        # Determine the longitude of the previous bridge
        lon_prev = df.iloc[i-1, df.columns.get_indexer(['lon'])].values
        # Check if the lat and lon of this bridge and the previous bridge are not the same
        if not (lat == lat_prev and lon == lon_prev):
            # if that is the case insert link
            # determine the length of the link by calculating the difference in km of the bridges
            # and substracting half of the length of the bridges.
            length = (1000*df.iloc[i, df.columns.get_indexer(['km'])].values 
                      - 1000*df.iloc[i-1, df.columns.get_indexer(['km'])].values 
                      - 0.5*df.iloc[i, df.columns.get_indexer(['length'])].values 
                      - 0.5*df.iloc[i-1, df.columns.get_indexer(['length'])].values)
            # Calculate the km of the link by taking the average
            km = (df.iloc[i, df.columns.get_indexer(['km'])].values[0]+df.iloc[i-1, df.columns.get_indexer(['km'])].values[0])/2
            # Calculate the latitude of the link by interpolating
            lat = (df.iloc[i, df.columns.get_indexer(['lat'])].values[0]+df.iloc[i-1, df.columns.get_indexer(['lat'])].values[0])/2
            # Calculate the longitude of the link by interpolating
            lon = (df.iloc[i, df.columns.get_indexer(['lon'])].values[0]+df.iloc[i-1, df.columns.get_indexer(['lon'])].values[0])/2
            # Add the link to the dictionary with the necessary attributes
            inserting_links[i] = {'road': ['N1'], 'km': [km], 'type': ['link'], 'name': ['link'], 'length': [round(length[0])], 'lat': [lat], 'lon': [lon]}
    return inserting_links

In [5]:
dict_links = create_links(df_N1_sorted, index)

In [6]:
# Create a function to insert the links into the main dataframe
def insert_links(df, dict_links):
    # Make a copy of the dataframe
    main_df = df.copy()
    # create a list of the indexes of the links to insert
    index_links = list(dict_links.keys())
    # Loop over the links in the dictionary
    for link in dict_links:
        # update the index by adding the index of the link in the list
        # so that the links that are already added to the dataframe are taken into account
        index = link + index_links.index(link)
        # create a dataframe based on the link we want to insert
        df_insert = pd.DataFrame(dict_links[link])
        # Add the link to the main dataframe by concatting the dataframe until the index,
        # the link dataframe and the dataframe from the index
        main_df = pd.concat([main_df.iloc[:index, ], df_insert, main_df.iloc[index:, ]])
    # Reset the index of the main dataframe
    main_df = main_df.reset_index(drop=True)
    # Return the dataframe
    return main_df

In [7]:
# Call the function to check the outcome
main_df = insert_links(df_N1_sorted, dict_links)
main_df

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.8000,Box Culvert,LRP001a,.,11.30,A,117861.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.800,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.698739,90.458861,interpolate
1,N1,3.3625,link,NaN,link,3116.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.696701,90.473318,NaN
2,N1,4.9250,Box Culvert,LRP004b,.,6.60,A,117862.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.694664,90.487775,interpolate
3,N1,6.9505,link,NaN,link,3851.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.699862,90.505495,NaN
4,N1,8.9760,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.705060,90.523214,interpolate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409,N1,459.7735,link,NaN,link,184.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.883819,92.298439,NaN
1410,N1,459.8660,Box Culvert,LRP456b,BUS STAND TEKNAF BOX CULVERT,1.50,C,100893.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,459.866,10.3,1989.0,1.0,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,20.883070,92.298163,interpolate
1411,N1,459.9895,link,NaN,link,228.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.882028,92.297970,NaN
1412,N1,460.1130,RCC Girder Bridge,LRP456c,YAKNAT BRIDGE,35.90,C,100889.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,460.113,9.2,1985.0,3.0,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,20.880985,92.297777,interpolate
